# Importing

In [1]:
import tensorflow
from tensorflow import keras
from keras import Sequential
from keras.layers import Dense,Flatten
from keras.applications.vgg16 import VGG16

In [2]:
conv_base = VGG16(
    weights='imagenet',
    include_top = False,
    input_shape=(150,150,3)
)



58889256/58889256 [==============================] - 57s 1us/step


In [3]:
conv_base.summary()

Model: "vgg16"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 150, 150, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 150, 150, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 150, 150, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 75, 75, 64)        0         
                                                                 
 block2_conv1 (Conv2D)       (None, 75, 75, 128)       73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 75, 75, 128)       147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 37, 37, 128)       0     

In [5]:
model = Sequential()

model.add(conv_base)
model.add(Flatten())
model.add(Dense(256,activation='relu'))
model.add(Dense(4,activation='softmax'))

In [6]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 vgg16 (Functional)          (None, 4, 4, 512)         14714688  
                                                                 
 flatten_1 (Flatten)         (None, 8192)              0         
                                                                 
 dense_2 (Dense)             (None, 256)               2097408   
                                                                 
 dense_3 (Dense)             (None, 4)                 1028      
                                                                 
Total params: 16813124 (64.14 MB)
Trainable params: 16813124 (64.14 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [7]:
conv_base.trainable = False

In [9]:
# generators
train_ds = keras.utils.image_dataset_from_directory(
    directory='../BrainTumor/Training',
    labels='inferred',
    label_mode='int',
    batch_size=32,
    image_size=(150, 150),
    # shuffle=True,
    # seed=42,
    # subset='training'  # Use this subset for training
)

validation_ds = keras.utils.image_dataset_from_directory(
    directory='../BrainTumor/Training',
    labels='inferred',
    label_mode='int',
    batch_size=32,
    image_size=(150, 150),
    # shuffle=True,
    # seed=42,
    # subset='validation'  # Use this subset for validation
)

Found 2870 files belonging to 4 classes.
Found 2870 files belonging to 4 classes.


In [10]:
# Normalize
def process(image,label):
    image = tensorflow.cast(image/255. ,tensorflow.float32)
    return image,label

train_ds = train_ds.map(process)
validation_ds = validation_ds.map(process)

In [11]:
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])


In [12]:
history = model.fit(train_ds,epochs=10,validation_data=validation_ds)

Epoch 1/10


90/90 [==============================] - 368s 4s/step - loss: 0.7421 - accuracy: 0.7449 - val_loss: 0.3116 - val_accuracy: 0.8969
Epoch 2/10
90/90 [==============================] - 344s 4s/step - loss: 0.3452 - accuracy: 0.8669 - val_loss: 0.2493 - val_accuracy: 0.9091
Epoch 3/10
90/90 [==============================] - 378s 4s/step - loss: 0.2300 - accuracy: 0.9181 - val_loss: 0.1347 - val_accuracy: 0.9599
Epoch 4/10
90/90 [==============================] - 355s 4s/step - loss: 0.1459 - accuracy: 0.9523 - val_loss: 0.0953 - val_accuracy: 0.9829
Epoch 5/10
90/90 [==============================] - 401s 4s/step - loss: 0.1451 - accuracy: 0.9509 - val_loss: 0.1024 - val_accuracy: 0.9645


KeyboardInterrupt: 